In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir



In [52]:
filepaths = [f for f in listdir("../data") if f.endswith('.xlsx')]


In [53]:
filepaths

['Returnee_MasterList_dataset_DTM_IOMJan 05, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMAug 31, 2019.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMMay 12, 2016.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMJan 19, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMOct 31, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMApr 10, 2015.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMJan 31, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMFeb 16, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMMar 02, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMFeb 29, 2020.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMApr 15, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMApr 29, 2020.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMFeb 15, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMNov 05, 2015.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMJul 31, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMSep 30, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMMar 30, 2017.xlsx',
 'Returnee_MasterList_dataset_D

In [61]:
df = pd.concat((pd.read_excel("../data/"+f).assign(date=f[35:47]) for f in filepaths))

In [73]:
df['date'] = pd.to_datetime(df['date'])